## Knowledge graphs visualizing patient diagnosis, prescription and procedures.

In [3]:
import networkx as nx
from pyvis.network import Network
import pandas as pd

patients_df = pd.read_csv('small_dataset/PATIENTS.csv')
admissions_df = pd.read_csv('small_dataset/ADMISSIONS.csv')
diagnoses_df = pd.read_csv('small_dataset/DIAGNOSES_ICD.csv')
procedures_df = pd.read_csv('small_dataset/PROCEDURES_ICD.csv')
prescriptions_df = pd.read_csv('small_dataset/PRESCRIPTIONS.csv')


first_patient_id = patients_df['subject_id'].iloc[0]

# Filter data for this patient
patients_df = patients_df[patients_df['subject_id'] == first_patient_id]
admissions_df = admissions_df[admissions_df['subject_id'] == first_patient_id]
diagnoses_df = diagnoses_df[diagnoses_df['hadm_id'].isin(admissions_df['hadm_id'])]
procedures_df = procedures_df[procedures_df['hadm_id'].isin(admissions_df['hadm_id'])]
prescriptions_df = prescriptions_df[prescriptions_df['hadm_id'].isin(admissions_df['hadm_id'])]

# Initialize the NetworkX graph
G = nx.DiGraph()

# Add nodes and edges for each entity type
# For Patients
G.add_node(str(first_patient_id), title=f"Patient {first_patient_id}", color="lightblue", size=25)

# For Admissions
for idx, row in admissions_df.iterrows():
    G.add_node(str(row['hadm_id']), title=f"Admission {row['hadm_id']}", color="yellow", size=20)
    G.add_edge(str(first_patient_id), str(row['hadm_id']), title='had admission')

# For Diagnoses
for idx, row in diagnoses_df.iterrows():
    G.add_node(str(row['icd9_code']), title=f"Diagnosis {row['icd9_code']}", color="green", size=15)
    G.add_edge(str(row['hadm_id']), str(row['icd9_code']), title='has diagnosis')

# For Procedures
for idx, row in procedures_df.iterrows():
    G.add_node(str(row['icd9_code']), title=f"Procedure {row['icd9_code']}", color="red", size=15)
    G.add_edge(str(row['hadm_id']), str(row['icd9_code']), title='underwent procedure')

# For Prescriptions
for idx, row in prescriptions_df.iterrows():
    G.add_node(str(row['drug']), title=f"Prescribed {row['drug']}", color="purple", size=15)
    G.add_edge(str(row['hadm_id']), str(row['drug']), title='was prescribed')

# Initialize PyVis network
nt = Network('1000px', '2000px', notebook=True)
nt.from_nx(G)
nt.show_buttons(filter_=['physics'])
nt.show("patient_journey.html")

patient_journey.html


## Visualizing the Diagnosis code and its meaning

In [4]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

# Load the dataset
diagnoses_df = pd.read_csv('small_dataset/D_ICD_DIAGNOSES.csv')

# Select only necessary columns and limit to the first 20 rows
diagnoses_df = diagnoses_df[['icd9_code', 'long_title']].head(20)

# Initialize the graph
G = nx.Graph()

# Add nodes and edges with simplified labels
for index, row in diagnoses_df.iterrows():
    icd_node = f"ICD9: {row['icd9_code']}"
    title_node = f"Title: {row['icd9_code']}"

    # Add nodes with the full long title as a hover tooltip
    G.add_node(icd_node, title=row['long_title'], label=icd_node, size=15)
    G.add_node(title_node, title=row['long_title'], label=title_node, size=15)
    
    # Add an edge between the ICD code and its title
    G.add_edge(icd_node, title_node, title="refers to", label="refers to", length=100, color='green')

# Create the network visualization
nt = Network('1000px', '2000px', notebook=True, directed=False)
nt.from_nx(G)

# Show the network
nt.show('diagnoses_network.html')


diagnoses_network.html


## Visualizing the Lab reports of patients and lab events

In [5]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

def generate_patient_network(patient_id):
    patients_df = pd.read_csv('small_dataset/PATIENTS.csv')
    admissions_df = pd.read_csv('small_dataset/ADMISSIONS.csv')
    labevents_df = pd.read_csv('small_dataset/LABEVENTS.csv')
    d_labitems_df = pd.read_csv('small_dataset/D_LABITEMS.csv')
    
    # Filter data for the given patient
    admissions_df_filtered = admissions_df[admissions_df['subject_id'] == patient_id]
    labevents_df_filtered = labevents_df[labevents_df['hadm_id'].isin(admissions_df_filtered['hadm_id'])]
    
    # Initialize the NetworkX graph
    G = nx.DiGraph()
    
    # Add node for the patient
    G.add_node(str(patient_id), title=f"Patient {patient_id}", color="lightblue", size=25)
    
    # For Admissions
    for idx, row in admissions_df_filtered.iterrows():
        G.add_node(str(row['hadm_id']), title=f"Admission {row['hadm_id']}", color="yellow", size=20)
        G.add_edge(str(patient_id), str(row['hadm_id']), title='had admission')
    
    # For Lab Events
    for idx, row in labevents_df_filtered.iterrows():
        item_desc = d_labitems_df[d_labitems_df['itemid'] == row['itemid']].iloc[0]['label']
        test_name = f"Test {row['itemid']} - {item_desc}"
        G.add_node(test_name, title=f"{test_name}: {row['value']} {row['valueuom']}", color="orange", size=15)
        G.add_edge(str(row['hadm_id']), test_name, title='lab result')
    
    # Initialize PyVis network
    nt = Network('1000px', '2000px', notebook=True)
    nt.from_nx(G)
    nt.show_buttons(filter_=['physics'])
    nt.show("lab_reports_network.html")

# Input patient ID
patient_id_input = int(input("Enter patient ID: ")) # check patients_csv and find subject_id attribute and get any id from it
generate_patient_network(patient_id_input)


Enter patient ID:  10038


lab_reports_network.html
